# Tabula Muris

## 소개

단일 세포 RNASeq 데이터셋을 처음부터 끝까지 분석하는 실습 경험을 제공하기 위해 [Tabula Muris](https://tabula-muris.ds.czbiohub.org/)의 데이터를 예시로 사용합니다. Tabula Muris는 모든 마우스 조직을 단일 세포 수준에서 프로파일링하려는 공동 노력의 결과물입니다. 전체 데이터셋에는 고처리량이지만 낮은 커버리지의 10X 데이터와 저처리량이지만 높은 커버리지의 Smartseq2 데이터가 모두 포함되어 있습니다.

이 워크숍에서는 마우스 뇌의 Smartseq2 데이터를 사용할 것입니다. 이 데이터는 다음으로 구성됩니다:  
1. 각 열이 유전자(또는 전사체)에 해당하고 각 행이 단일 세포에 해당하는 __발현 행렬__  
2. 각 세포를 설명하는 __메타데이터__ 표

## 데이터 다운로드

데이터는 이 강의 리포지토리의 `scRNA-python-workshop/content/data.zip`에 번들로 제공됩니다. [여기](https://github.com/chanzuckerberg/scRNA-python-workshop/blob/master/content/data.zip)에서 데이터 파일을 직접 찾을 수도 있습니다. 폴더의 압축을 풀어 파일이 `scRNA-python-workshop/content/data/` 디렉토리에 위치하도록 하십시오.

## 데이터 읽기

이제 쉼표로 구분된 파일에서 카운트 행렬을 읽어올 수 있습니다. 그런 다음 결과 데이터프레임을 검사합니다:

In [1]:
import pandas as pd ## 데이터프레임 작업을 위한 pandas 라이브러리 로드

In [3]:
## pandas에게 `brain_counts.csv`의 내용으로 새 DataFrame을 만들라고 지시합니다. 이 작업은 1분 정도 걸릴 수 있습니다.
count_dataframe = pd.read_csv('../data/brain_counts.csv', # 데이터를 찾을 위치
                              index_col=0) # 첫 번째 열을 사용하여 행 레이블 지정('인덱스')

## 데이터프레임의 첫 2개 행 출력
print(count_dataframe.head(2)) 

                       0610005C13Rik  0610007C21Rik  0610007L01Rik  \
A1.B003290.3_38_F.1.1              0            125             16   
A1.B003728.3_56_F.1.1              0              0              0   

                       0610007N19Rik  0610007P08Rik  0610007P14Rik  \
A1.B003290.3_38_F.1.1              0              0              0   
A1.B003728.3_56_F.1.1              0              0            324   

                       0610007P22Rik  0610008F07Rik  0610009B14Rik  \
A1.B003290.3_38_F.1.1              0              0              0   
A1.B003728.3_56_F.1.1              0              0              0   

                       0610009B22Rik  ...  Zxdb  Zxdc  Zyg11a  Zyg11b  Zyx  \
A1.B003290.3_38_F.1.1              0  ...     0     0       0       0    0   
A1.B003728.3_56_F.1.1              0  ...     0     0       0       0    0   

                       Zzef1  Zzz3  a  l7Rn6  zsGreen_transgene  
A1.B003290.3_38_F.1.1      0     0  0     54                  0  


### 연습  

열 이름은 무엇을 나타냅니까? 행 이름은 무엇을 나타냅니까? 이 데이터셋에는 몇 개의 세포와 유전자가 있습니까?

힌트: 도움이 필요하십니까? `help(pd.DataFrame.shape)`를 시도해 보십시오.

<p>
<details>
<summary><h3>해결책</h3></summary>
열 이름은 유전자를 나타냅니다. 
행 이름은 데이터셋 작성자가 할당한 고유한 세포 식별자를 나타냅니다.
<p/>
<code>shape</code>를 요청하여 데이터셋에 몇 개의 유전자와 세포가 있는지 알아낼 수 있습니다:  <br>
<code>print(count_dataframe.shape)</code><br>
결과:<br>
<code>(3401, 23433)</code>
<p/>
이는 <code>(행 수, 열 수)</code>를 나타냅니다.
</details>
</p>

In [ ]:
help(pd.DataFrame.shape)

## 메타데이터 읽기

작성자들은 각 세포를 설명하는 메타데이터도 제공했습니다. 이 메타데이터는 별도의 파일인 `brain_metadata.csv`에 저장됩니다. 카운트 데이터와 마찬가지로 데이터프레임으로 로드하여 검사할 수 있습니다. 

### 연습  

csv 파일에서 메타데이터를 `metadata_dataframe`이라는 pandas 데이터프레임으로 로드합니다. `counts_dataframe`과 동일한 차원과 인덱스를 가지고 있습니까?

<p>
<details>
<summary><h3>해결책</h3></summary>
먼저 메타데이터를 로드해야 합니다.<br>
<code style=display:block;white-space:pre-wrap>metadata_dataframe = pd.read_csv('../data/brain_metadata.csv', # 데이터를 찾을 위치
                           index_col=0) # 첫 번째 열을 인덱스로 사용</code>
    
<p>
결과 데이터프레임이 올바르게 보이는지 확인하기 위해 살펴보겠습니다. <br>
<code style=display:block;white-space:pre-wrap>print(metadata_dataframe.shape)
>>> (3401, 5)</code><br>
3,401개 세포에 대한 5개 열의 정보가 있습니다. 합리적으로 들립니다.
    
</p><p>
처음 몇 행을 자세히 살펴보겠습니다:<p/>
<code>print(metadata_dataframe.head())</code><br>
<code style=display:block;white-space:pre-wrap>>>>                        cell_ontology_class    subtissue mouse.sex mouse.id  \
cell                                                                          
A1.B003290.3_38_F.1.1             astrocyte     Striatum         F   3_38_F   
A1.B003728.3_56_F.1.1             astrocyte     Striatum         F   3_56_F   
A1.MAA000560.3_10_M.1.1     oligodendrocyte       Cortex         M   3_10_M   
A1.MAA000564.3_10_M.1.1    endothelial cell     Striatum         M   3_10_M   
A1.MAA000923.3_9_M.1.1            astrocyte  Hippocampus         M    3_9_M   
                        plate.barcode  
cell                                   
A1.B003290.3_38_F.1.1         B003290  
A1.B003728.3_56_F.1.1         B003728  
A1.MAA000560.3_10_M.1.1     MAA000560  
A1.MAA000564.3_10_M.1.1     MAA000564  
A1.MAA000923.3_9_M.1.1      MAA000923</code>
    </p>
</details>
</p>

In [3]:
metadata_dataframe = pd.read_csv()



### 연습  

이 데이터셋에 무엇이 있는지 파악하기 위해 각 메타데이터 열의 요약을 살펴보겠습니다. 

1. 이 데이터셋에는 각 `subtissue` 레이블의 세포가 몇 개 있습니까? <br>
힌트: `help(pd.value_counts)`를 실행하여 시작해 보십시오.  
<p>
2. `for` 루프를 사용하여 이 계산 절차를 반복하여 각 메타데이터 열을 요약합니다.  <br>
힌트: `columns = metadata_dataframe.columns.values`로 데이터프레임의 모든 열 이름에 액세스할 수 있습니다.  


<p>
<details>
<summary><h3>해결책</h3></summary>
1. 열에 각 값이 나타나는 횟수를 다음과 같이 계산할 수 있습니다:<br>
    <code style=display:block;white-space:pre-wrap>print(pd.value_counts(metadata_dataframe['subtissue']))
>>>>        
Cortex         1149
Hippocampus     976
Striatum        723
Cerebellum      553
Name: subtissue, dtype: int64</code>
    <p/>
2. 데이터프레임의 각 열에 대해 이 작업을 반복하려면 <code>for</code> 루프를 사용할 수 있습니다:<br>
<code style=display:block;white-space:pre-wrap>for column in metadata_dataframe.columns.values:
    print(pd.value_counts(metadata_dataframe[column]))</code>
</details>
    </p>

## AnnData 객체 빌드

이제 Tabula Muris 뇌 데이터셋의 카운트와 메타데이터를 포함하는 두 개의 데이터프레임이 있습니다. 이를 정리하기 위해 **AnnData**라는 **데이터 구조**를 사용합니다. AnnData는 "annotated data"의 약자이며 분석 라이브러리인 **SCANPY**에서 사용하는 표준 형식입니다. 

AnnData는 세포와 유전자를 설명하기 위해 일반화된 어휘를 사용합니다. 세포를 **관측치(observations)**, 유전자를 **변수(variables)**라고 합니다. 이 데이터 구조에는 정보를 저장할 수 있는 네 가지 영역이 있습니다:  

<img src="../figures/anndata.svg" alt="Drawing" style="width: 500px;"/>

`AnnData.X`는 카운트 행렬을 저장합니다.  
`AnnData.obs`는 관측치(세포)에 대한 메타데이터를 저장합니다.  
`AnnData.var`는 변수(유전자)에 대한 메타데이터를 저장합니다.  
`AnnData.uns`는 나중에 첨부하기로 결정한 추가적인 비정형 정보를 저장합니다.  



여기에는 각 세포를 설명하는 카운트 행렬과 메타데이터가 있으므로 AnnData 구조의 `.X` 및 `.obs` 부분을 사용합니다. 

In [4]:
import scanpy as sc # AnnData 데이터 구조가 어떻게 작동하는지 Python에 알려주는 scanpy 라이브러리 가져오기

In [5]:
# help(sc.AnnData)
adata = sc.AnnData(X = count_dataframe, obs = metadata_dataframe)
print(adata)

AnnData object with n_obs × n_vars = 3401 × 23433 
    obs: 'cell_ontology_class', 'subtissue', 'mouse.sex', 'mouse.id', 'plate.barcode'


### 스파이크인 라벨링
이것은 smartseq2 데이터이므로 스파이크인이 있을 수 있습니다. 이러한 유전자 이름은 `ERCC`로 시작합니다. `adata.var`에서 유전자 주석으로 라벨을 지정할 수 있습니다.

In [6]:
is_spike_in = {}
number_of_spike_ins = 0

for gene_name in adata.var_names:
    if 'ERCC' in gene_name:
        is_spike_in[gene_name] = True # 스파이크인을 찾았음을 기록
        number_of_spike_ins += 1 # 카운터 증가
    else:
        is_spike_in[gene_name] = False # 스파이크인이 아님을 기록
        
adata.var['ERCC'] = pd.Series(is_spike_in) # adata.var의 인덱스와 is_spike_in의 키가 일치하므로 anndata가 일치하도록 처리합니다.
print('발견된 스파이크인 수: ', number_of_spike_ins)

found this many spike ins:  92


이제 AnnData 객체 빌드를 마쳤으므로 나중에 사용하기 위해 다음과 같이 파일에 저장할 수 있습니다:

In [8]:
adata.write('../data/brain_raw.h5ad') ## h5ad 확장자는 AnnData 전용입니다.

... storing 'cell_ontology_class' as categorical
... storing 'subtissue' as categorical
... storing 'mouse.sex' as categorical
... storing 'mouse.id' as categorical
... storing 'plate.barcode' as categorical
